Detection Movement Function

In [7]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

import cv2
import numpy as np
import time
from PIL import Image
import imutils

from IPython.display import display
import ipywidgets.widgets as widgets

In [2]:
# Init camera
camera = Camera.instance()

In [3]:
def motion_detection(firstFrame, camera):
    """
    Detect motion by comparing the current camera frame with a firstFrame.
    
    Arguments:
        firstFrame (array): first frame considered as the baseline
        camera (Camera): instantiated Jetbot camera, used to obtain new frames.
        
    Returns:
        list: of regions where movement has been detected (left, top, right, bottom)
    """
    # Adapted from https://www.pyimagesearch.com/2015/05/25/basic-motion-detection-and-tracking-with-python-and-opencv/

    # Min movement area
    MIN_AREA = 50

    # Gaussian blur kernel
    KERNEL_SIZE = 3

    # Movement hreshold
    THRESHOLD = 37

    # List to store regions where movement is detected
    motion_bboxes = []
    
    # firstFrame preprocessing
    firstFrame = cv2.cvtColor(firstFrame, cv2.COLOR_BGR2GRAY)
    firstFrame = cv2.GaussianBlur(firstFrame, (KERNEL_SIZE, KERNEL_SIZE), 0)

    ff_widget.value = bgr8_to_jpeg(firstFrame)
    
    # grab the current frame and initialize the static/moving text
    frame = camera.value.copy()

    # convert frame to grayscale, and blur it
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (KERNEL_SIZE, KERNEL_SIZE), 0)

    # compute the absolute difference between the current frame and
    # first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, THRESHOLD, 255, cv2.THRESH_BINARY)[1]

    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)   
    cnts = imutils.grab_contours(cnts)

    # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < MIN_AREA:
            continue

        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
        
        motion_bboxes.append((x, y, x+w, y+h))    # (left, top, right, bottom)

    # Display image frame
    image_widget.value = bgr8_to_jpeg(frame)
    dif_widget.value = bgr8_to_jpeg(thresh)
    
    return motion_bboxes
    

In [4]:
# Image widget
image_widget = widgets.Image(format='jpeg', width=400, height=400)
dif_widget = widgets.Image(format='jpeg', width=400, height=400)
ff_widget = widgets.Image(format='jpeg', width=400, height=400)

display(widgets.HBox([ff_widget, image_widget, dif_widget]))

In [15]:
time.sleep(3)

MAX_TIME = 10

first_frame = camera.value.copy()

motion_boxes = []
t0 = time.time()
elapsed_time = 0

# Run motion detection for MAX_TIME
while elapsed_time < MAX_TIME:
    m_bboxes = motion_detection(first_frame, camera)
    motion_boxes.extend(m_bboxes)
    
    elapsed_time =  time.time() - t0
    
motion_boxes = set(motion_boxes)    # remove duplicates

In [13]:
def box_in_box(out_box, in_box, margin=0):
    """
    Evaluate if a bounding box is partially inside another one.
    
    Arguments:
        out_box (array): outter bounding box (left, top, right, bottom).
        in_box (array): inner bounding box (left, top, right, bottom).
        margin (int, optional): percentual margin to add to outter box. Defaults to 0.
        
    Returns:
        boolean
    """
    top_left = False
    top_right = False
    
    # Apply margin
    out_box[0] = int((1 - margin/100) * out_box[0])
    out_box[1] = int((1 - margin/100) * out_box[1])
    out_box[2] = int((1 + margin/100) * out_box[2])
    out_box[3] = int((1 + margin/100) * out_box[3])
    
    # Top-left corner
    if out_box[0] <= in_box[0] <= out_box[2] and out_box[1] <= in_box[1] <= out_box[3]:
        top_left = True
        print('top_left_corner True')
        
    # Bottom-right corner
    if out_box[0] <= in_box[2] <= out_box[2] and out_box[1] <= in_box[3] <= out_box[3]:
        bottom_right = True
        print('bottom_right_corner True')
            
    # Return True if top-left and bottom-right corners are inside
    if top_left and bottom_right:
        return True